In [3]:
import json
import random

# Carica il dataset originale (qui simulo una singola riga come esempio)
with open("/home/dtesta/XAI-VLMs-MC_consistency/MAIA_rows.jsonl", "r", encoding="utf-8") as f:
    original_data = json.load(f)

new_dataset = []

for item in original_data:
    video_id = item["video_id"]
    category = item["category"]
    cat_split = item["cat_split"]
    captions = item["captions"]
    foils = item["foils"]

    for i in range(8):
        caption = captions[i]
        foil = foils[i]
        pool = i + 1

        # Randomizza l'ordine tra caption e foil nel prompt
        if random.random() < 0.5:
            prompt = f"Scegli la descrizione corretta:\nA. {caption}\nB. {foil}\nRispondi solo A o B"
            target = 0
        else:
            prompt = f"Scegli la descrizione corretta:\nA. {foil}\nB. {caption}\nRispondi solo A o B"
            target = 1

        new_entry = {
            "video_id": video_id,
            "category": category,
            "cat_split": cat_split,
            "caption": caption,
            "foil": foil,
            "pool": pool,
            "prompt": prompt,
            "target": target
        }

        new_dataset.append(new_entry)

# Salva il nuovo dataset
with open("caption_foil_prompt_dataset.json", "w", encoding="utf-8") as f:
    json.dump(new_dataset, f, ensure_ascii=False, indent=2)


JSONDecodeError: Extra data: line 2 column 1 (char 2020)

In [4]:
import json
import random
from pathlib import Path

# Load the original JSON file
input_path = Path("/home/dtesta/XAI-VLMs-MC_consistency/MAIA_grouped.json")
with input_path.open("r", encoding="utf-8") as f:
    data = json.load(f)

# Prepare the transformed data
transformed_data = []

for video in data:
    video_id = video.get("video")
    link = video.get("link")
    
    for group_key in ["question_categories_A", "question_categories_B"]:
        for qa in video.get(group_key, []):
            category = qa.get("category")
            question = qa.get("question")
            answers = qa.get("answer", [])
            captions = qa.get("caption", [])
            foils = qa.get("foil", [])
            
            # Process each of the 8 items
            for i in range(min(8, len(captions), len(foils))):
                caption = captions[i]
                foil = foils[i]
                
                # Randomly assign caption and foil to A or B
                if random.random() < 0.5:
                    prompt = f"Scegli la descrizione corretta:\n A. {caption}.\n B. {foil}.\nRispondi solo A o B.\n"
                    caption_position = 1
                else:
                    prompt = f"Scegli la descrizione corretta:\n A. {foil}.\n B. {caption}.\nRispondi solo A o B.\n"
                    caption_position = 0
                
                item = {
                    "video_id": video_id,
                    "link": link,
                    "category": category,
                    "question": question,
                    "answer": answers[i] if i < len(answers) else None,
                    "caption": caption,
                    "foil": foil,
                    #"item_pool_number": f"{video_id}_pool_{i + 1}",
                    "item_pool_number": i + 1,
                    "multiple_choice_prompt": prompt,
                    "caption_position": caption_position
                }
                transformed_data.append(item)

# Save to a new JSON file
output_path = "/home/dtesta/XAI-VLMs-MC_consistency/MAIA_flattened_items.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(transformed_data, f, indent=2, ensure_ascii=False)

output_path


'/home/dtesta/XAI-VLMs-MC_consistency/MAIA_flattened_items.json'

In [9]:
import json

with open("/home/dtesta/XAI-VLMs-MC_consistency/MAIA_flattened_items.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print("Numero totale di item:", len(data))
# Conta gli item con 'video1' nel video_id
count = sum(1 for item in data if item.get('video_id') == 'video100')

print(f"Numero di item con video_id 'video1': {count}")


Numero totale di item: 19200
Numero di item con video_id 'video1': 192
